# Check RHS Expression of cGHG system

In [2]:
Needs["xAct`xCoba`", FileNameJoin[{Environment["GENERATO"], "src/Generato.wl"}]]

------------------------------------------------------------
Package xAct`xPerm`  version 1.2.3, {2015, 8, 23}
CopyRight (C) 2003-2020, Jose M. Martin-Garcia, under the General Public\
 
>   License.
Connecting to external mac executable...
Connection established.
------------------------------------------------------------
Package xAct`xTensor`  version 1.2.0, {2021, 10, 17}
CopyRight (C) 2002-2021, Jose M. Martin-Garcia, under the General Public\
 
>   License.
------------------------------------------------------------
These packages come with ABSOLUTELY NO WARRANTY; for details type\
 
>   Disclaimer[]. This is free software, and you are welcome to redistribute\
 
>   it under certain conditions. See the General Public License for details.
------------------------------------------------------------
------------------------------------------------------------
Package xAct`xCoba`  version 0.8.6, {2021, 2, 28}
CopyRight (C) 2005-2021, David Yllanes and Jose M. Martin-Garcia, under t

In [3]:
DefManifold[M4, 4, IndexRange[a, z]];

In [4]:
DefChart[cart, M4, {0, 1, 2, 3}, {T[], X[], Y[], Z[]}, ChartColor -> Blue];

In [5]:
<< "wl/cGHG_vars_check.wl";

In [6]:
<< "wl/ADM_vars_check.wl";

In [7]:
<< "wl/cGHG_rhs.wl";

## Define Scalar Functions

In [ ]:
DefScalarFunction[fW];
Module[{co = {x, y, z}},
  DefScalarFunction[falpha];
  Do[DefScalarFunction[ToExpression["fbeta" <> ToString[co[[ii]]]]], {ii, 1, 3}];
  Do[DefScalarFunction[ToExpression["fgamma" <> ToString[co[[ii]]] <> ToString[co[[jj]]]]], {ii, 1, 3}, {jj, ii, 3}];
  Do[DefScalarFunction[ToExpression["fK" <> ToString[co[[ii]]] <> ToString[co[[jj]]]]], {ii, 1, 3}, {jj, ii, 3}]
];
Module[{co = {x, y, z}},
  DefScalarFunction[fdtalpha];
  Do[DefScalarFunction[ToExpression["fdtbeta" <> ToString[co[[ii]]]]], {ii, 1, 3}];
  Do[DefScalarFunction[ToExpression["fdtgamma" <> ToString[co[[ii]]] <> ToString[co[[jj]]]]], {ii, 1, 3}, {jj, ii, 3}]
];

## Set ADM Variable Components with Scalar Functions

In [ ]:
ComponentValue[W[], fW[X[], Y[], Z[]]];

In [ ]:
Module[{co = {x, y, z}},
  ComponentValue[ADMalpha[], falpha[X[], Y[], Z[]]];
  Do[ComponentValue[ADMbeta[{ii, cart}], If[ii == 0, 0, ToExpression["fbeta" <> ToString[co[[ii]]] <> "[X[],Y[],Z[]]"]]], {ii, 0, 3}];
  Do[ComponentValue[ADMgamma[{ii, -cart}, {jj, -cart}], ToExpression["fgamma" <> ToString[co[[ii]]] <> ToString[co[[jj]]] <> "[X[],Y[],Z[]]"]], {ii, 1, 3}, {jj, ii, 3}];
  Do[ComponentValue[ADMexK[{ii, -cart}, {jj, -cart}], ToExpression["fK" <> ToString[co[[ii]]] <> ToString[co[[jj]]] <> "[X[],Y[],Z[]]"]], {ii, 1, 3}, {jj, ii, 3}]
];
ADMgamma[-i, -j] // ToBasis[cart] // ComponentArray // ComponentValue;
ADMexK[-i, -j] // ToBasis[cart] // ComponentArray // ComponentValue;

In [ ]:
Module[{co = {x, y, z}},
  ComponentValue[ADMdtalpha[], fdtalpha[X[], Y[], Z[]]];
  Do[ComponentValue[ADMdtbeta[{ii, cart}], If[ii == 0, 0, ToExpression["fdtbeta" <> ToString[co[[ii]]] <> "[X[],Y[],Z[]]"]]], {ii, 0, 3}];
  Do[ComponentValue[ADMdtgamma[{ii, -cart}, {jj, -cart}], ToExpression["fdtgamma" <> ToString[co[[ii]]] <> ToString[co[[jj]]] <> "[X[],Y[],Z[]]"]], {ii, 1, 3}, {jj, ii, 3}]
];
ADMdtgamma[-i, -j] // ToBasis[cart] // ComponentArray // ComponentValue;

## Set cGHG Variables with ADM Variables

$\alpha,\beta^i$

In [ ]:
ComponentValue[alpha[], ADMalpha[] // ToValues];
Do[ComponentValue[beta[{ii, cart}], ADMbeta[{ii, cart}] // ToValues], {ii, 0, 3}];

$\gamma^{ij}=(\gamma_{ij})^{-1}, \hat\gamma^{ij}=W^{-2}\gamma^{ij}$ where $W=(\gamma/\hat{g})^{-1/6}=\gamma^{-1/6}$

In [ ]:
Module[{gammaMat, invgammaMat},
  gammaMat = Table[ADMgamma[{ii, -cart}, {jj, -cart}] // ToValues, {ii, 1, 3}, {jj, 1, 3}];
  invgammaMat = Inverse[gammaMat] /. {1/Det[gammaMat] -> (W[] // ToValues)^6};
  Do[ComponentValue[invgam[{aa, cart}, {bb, cart}], If[aa == 0 || bb == 0, 0, invgammaMat[[aa, bb]]]], {aa, 0, 3}, {bb, 0, 3}]
];
Do[ComponentValue[invhg[{aa, cart}, {bb, cart}], If[aa == 0 || bb == 0, 0, (W[] // ToValues)^-2 invgam[{aa, cart}, {bb, cart}] // ToValues]], {aa, 0, 3}, {bb, 0, 3}];

$n^a = (\alpha^{-1},-\alpha^{-1}\beta^i)$

In [ ]:
Do[ComponentValue[nvec[{aa, cart}], If[aa == 0, ADMalpha[]^-1, -ADMalpha[]^-1 ADMbeta[{aa, cart}]] // ToValues], {aa, 0, 3}];

$g^{ab}=\gamma^{ab}-n^an^b$

In [ ]:
Do[ComponentValue[invg4[{aa, cart}, {bb, cart}], invgam[{aa, cart}, {bb, cart}] - nvec[{aa, cart}] nvec[{bb, cart}] // ToValues], {aa, 0, 3}, {bb, 0, 3}];b

$\partial_cg_{ij}=\partial_c\gamma_{ij}, \partial_cg_{0i}=\partial_c\gamma_{il}\beta^l+\gamma_{il}\partial_c\beta^l, \partial_cg_{00}=-2\alpha\partial_c\alpha+2\beta_l\partial_c\beta^l+\beta^k\beta^l\partial_c\gamma_{kl}$

In [ ]:
Do[ComponentValue[dg4[{cc, -cart}, {aa, -cart}, {bb, -cart}],
    If[cc == 0,
        If[aa == 0 && bb == 0,
            -2 ADMalpha[] ADMdtalpha[] + 2 ADMgamma[-k, -l] ADMbeta[k] ADMdtbeta[l] + ADMbeta[k] ADMbeta[l] ADMdtgamma[-k, -l],
            If[aa == 0,
                ADMbeta[l] ADMdtgamma[{bb, -cart}, -l] + ADMgamma[{bb, -cart}, -l] ADMdtbeta[l],
                ADMdtgamma[{aa, -cart}, {bb, -cart}]
            ]
        ],
        If[aa == 0 && bb == 0,
            -2 ADMalpha[] PDcart[{cc, -cart}][ADMalpha[]] + 2 ADMgamma[-k, -l] ADMbeta[k] PDcart[{cc, -cart}][ADMbeta[l]] + ADMbeta[k] ADMbeta[l] PDcart[{cc, -cart}][ADMgamma[-k, -l]],
            If[aa == 0,
                ADMbeta[l] PDcart[{cc, -cart}][ADMgamma[{bb, -cart}, -l]] + ADMgamma[{bb, -cart}, -l] PDcart[{cc, -cart}][ADMbeta[l]],
                PDcart[{cc, -cart}][ADMgamma[{aa, -cart}, {bb, -cart}]]
            ]
        ]
    ] // DummyToBasis[cart] // TraceBasisDummy // ToValues
], {cc, 0, 3}, {aa, 0, 3}, {bb, aa, 3}];

In [ ]:
dg4[-c, -a, -b] // ToBasis[cart] // ComponentArray // ComponentValue;